In [148]:
import pandas as pd

df = pd.read_csv('LUSC_nodes.csv')
df.head(10)

,id,x1,x2
0,MIR4640///DDR1,9.185,10.529
1,RFC2,6.282,6.803
2,HSPA6,6.508,6.514
3,PAX8,8.945,8.898
4,GUCA1A,3.974,4.007
5,MIR5193///UBA7,8.272,7.508
6,THRA,5.734,5.851
7,PTPN21,5.328,4.861
8,CCL5,7.658,6.920
9,CYP2E1,4.059,4.133


In [149]:
preprs = pd.notnull(df["id"])
preprs

# displayed data only with team = NaN
data = df[preprs]
data

,id,x1,x2
0,MIR4640///DDR1,9.185,10.529
1,RFC2,6.282,6.803
2,HSPA6,6.508,6.514
3,PAX8,8.945,8.898
4,GUCA1A,3.974,4.007
...,...,...,...
22232,GAPDH,11.811,12.896
22233,STAT1,8.597,8.684
22234,STAT1,4.482,4.619
22235,STAT1,7.656,7.666


In [158]:
#filtered = data.loc[data.id.str.contains("///")]
#filtered

#data_new = data[~data["id"].str.contains('///')]
#data_new

data

,id,x1,x2
0,MIR4640///DDR1,9.185,10.529
1,RFC2,6.282,6.803
2,HSPA6,6.508,6.514
3,PAX8,8.945,8.898
4,GUCA1A,3.974,4.007
...,...,...,...
22232,GAPDH,11.811,12.896
22233,STAT1,8.597,8.684
22234,STAT1,4.482,4.619
22235,STAT1,7.656,7.666


In [162]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

newdata = explode(data.assign(id=data.id.str.split('///')), 'id')

In [163]:
newdata

,x1,x2,id
0,9.185,10.529,MIR4640
1,9.185,10.529,DDR1
2,6.282,6.803,RFC2
3,6.508,6.514,HSPA6
4,8.945,8.898,PAX8
...,...,...,...
23392,11.811,12.896,GAPDH
23393,8.597,8.684,STAT1
23394,4.482,4.619,STAT1
23395,7.656,7.666,STAT1


In [164]:
newdata.to_csv('data_modified.csv')

TypeError: object of type 'float' has no len()